In [14]:
import pyspark.pandas as ps
from pyspark.sql import SparkSession
import pandas as pd

In [15]:
spark = SparkSession.builder.config("spark.executor.memory", "10g").config("spark.driver.memory", "2G").getOrCreate()

In [16]:
data = spark.read.csv("./부동산실거래가1998-2023.csv", header=True)

In [17]:
data.show()

+---------------------------------+-----------+------------+--------+--------------+---+--------+
|                           시군구|     단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|
+---------------------------------+-----------+------------+--------+--------------+---+--------+
|강원특별자치도 양구군 양구읍 상리|       경림|        59.4|  200509|          5000| 11|    1998|
|           경기도 의정부시 민락동| 한라비발디|       84.99|  200510|         12750| 19|    2003|
|           경기도 의정부시 신곡동|      신일1|      59.878|  200510|          5750|  3|    1997|
|           경기도 의정부시 신곡동|       풍림|       49.83|  200510|          5500|  8|    1998|
|           경기도 의정부시 용현동|용현현대1차|      129.73|  200511|         12200| 11|    1992|
|           경기도 의정부시 용현동|용현현대1차|      129.73|  200511|         11500| 17|    1992|
|     강원특별자치도 강릉시 견소동|   송정한신|       43.38|  200601|          4200|  1|    1997|
|       강원특별자치도 강릉시 교동|  교동1주공|       59.89|  200601|          7500|  2|    1999|
|       강원특별자치도 강릉시 교동|  교동1주공|       59.89|  200601|          8500

In [18]:
data.printSchema()

root
 |-- 시군구: string (nullable = true)
 |-- 단지명: string (nullable = true)
 |-- 전용면적(㎡): string (nullable = true)
 |-- 계약년월: string (nullable = true)
 |-- 거래금액(만원): string (nullable = true)
 |-- 층: string (nullable = true)
 |-- 건축년도: string (nullable = true)



# 전체 데이터의 양 측정
* count()= 전체 행
* len(데이터프레임.columns) = 컬럼 수 

In [19]:
import pyspark.sql.functions as f

In [20]:
print(data.count(), len(data.columns))

9591228 7


# 컬럼의 데이터 타입 바꾸기 (거래금액, 전용면적) 숫자로 바꾸기

In [21]:
data = data.withColumn('거래금액(만원)', f.col('거래금액(만원)').cast('int'))

In [22]:
data.printSchema()

root
 |-- 시군구: string (nullable = true)
 |-- 단지명: string (nullable = true)
 |-- 전용면적(㎡): string (nullable = true)
 |-- 계약년월: string (nullable = true)
 |-- 거래금액(만원): integer (nullable = true)
 |-- 층: string (nullable = true)
 |-- 건축년도: string (nullable = true)



In [23]:
data = data.withColumn('전용면적(㎡)', f.col('전용면적(㎡)').cast('float'))

In [24]:
data.printSchema()

root
 |-- 시군구: string (nullable = true)
 |-- 단지명: string (nullable = true)
 |-- 전용면적(㎡): float (nullable = true)
 |-- 계약년월: string (nullable = true)
 |-- 거래금액(만원): integer (nullable = true)
 |-- 층: string (nullable = true)
 |-- 건축년도: string (nullable = true)



In [25]:
data.summary().show()

+-------+-----------------------------------+---------------+------------------+------------------+------------------+-----------------+------------------+
|summary|                             시군구|         단지명|      전용면적(㎡)|          계약년월|    거래금액(만원)|               층|          건축년도|
+-------+-----------------------------------+---------------+------------------+------------------+------------------+-----------------+------------------+
|  count|                            9591228|        9591228|           9591228|           9591228|           9591228|          9591228|           9591228|
|   mean|                               null|           null| 74.46021320549711|201434.57845356193|26051.670125660658|8.759394521744245|2000.0007455771045|
| stddev|                               null|           null|26.196316055716533| 503.5909999681341|25687.646317538885|6.020988738465453| 8.664532884432418|
|    min|강원특별자치도 강릉시 강동면 안인리|         (1-10)|              9.26|            200509|       

In [28]:
# 결측값 검사
data.select(*(f.sum(f.col(c).isNull().cast('int')).alias(c) for c in data.columns)).show()

+------+------+------------+--------+--------------+---+--------+
|시군구|단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|
+------+------+------------+--------+--------------+---+--------+
|     0|     0|           0|       0|             0|  0|       0|
+------+------+------------+--------+--------------+---+--------+



# 데이터 프레임에서 이상값 찾기

In [31]:
bounds = data.stat.approxQuantile("거래금액(만원)", [0.25, 0.75], 0)

In [32]:
bounds

[11000.0, 32000.0]

In [33]:
# 이상치 최대값, 최소값
lower_range = bounds[0] - (1.5 * (bounds[1] - bounds[0]))
upper_range = bounds[1] + (1.5 * (bounds[1] - bounds[0]))


In [34]:
print(lower_range)
print(upper_range)

-20500.0
63500.0


In [38]:
data.filter((data['거래금액(만원)'] < lower_range) | (data['거래금액(만원)'] > upper_range)).count()

578946

In [39]:
data.filter((data['거래금액(만원)'] < lower_range) | (data['거래금액(만원)'] > upper_range)).show()

+--------------------------+---------------+------------+--------+--------------+---+--------+
|                    시군구|         단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|
+--------------------------+---------------+------------+--------+--------------+---+--------+
|경기도 고양일산동구 마두동| 강촌마을(영남)|      165.99|  200601|         78000| 18|    1992|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      134.99|  200601|         71300| 10|    1993|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      134.99|  200601|         73000|  5|    1993|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      134.99|  200601|         67000|  7|    1993|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      164.37|  200601|         87500| 17|    1993|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      164.37|  200601|        102750|  5|    1993|
|경기도 고양일산동구 마두동| 강촌마을(한신)|      173.58|  200601|         78500| 17|    1993|
|경기도 고양일산동구 마두동| 강촌마을(한신)|      173.58|  200601|         73500| 12|    1993|
|경기도 고양일산동구 마두동| 강촌마을(한신)|     134.803|  200601|         67500| 10|    1993|
|경기도 고양일산동구 마두동| 백마마을(극동)|       132.8|  200601|

# 데이터프레임에서 이상값 찾는 함수

In [40]:
data.select('전용면적(㎡)').dtypes

[('전용면적(㎡)', 'float')]

In [42]:
data.select('전용면적(㎡)').dtypes[0][1]

'float'

In [46]:
def out_lier(data):
    import pyspark.sql.functions as f
    for col_name in data.columns:
        if data.select(col_name).dtypes[0][1] in ('int', 'integer', 'float', 'float32', 'float64', 'double'):
            bounds = data.stat.approxQuantile(col_name, [0.25, 0.75], 0)
            # 이상치 범위 계산
            lower_range = bounds[0] - (1.5 * (bounds[1] - bounds[0]))
            upper_range = bounds[1] + (1.5 * (bounds[1] - bounds[0]))
            print(col_name, lower_range, upper_range)
            # 이상치 필터링
            outliers = data.filter((data[col_name] < lower_range) | (data[col_name] > upper_range))
            print("number of outliers: ", outliers.count())
            outliers.show()

In [47]:
out_lier(data)

전용면적(㎡) 21.3750057220459 123.09499168395996


number of outliers:  639736
+-----------------------------------+-------------------+------------+--------+--------------+---+--------+
|                             시군구|             단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|
+-----------------------------------+-------------------+------------+--------+--------------+---+--------+
|             경기도 의정부시 용현동|        용현현대1차|      129.73|  200511|         12200| 11|    1992|
|             경기도 의정부시 용현동|        용현현대1차|      129.73|  200511|         11500| 17|    1992|
|       강원특별자치도 강릉시 노암동|           노암한라|     129.801|  200601|         11300|  6|    1994|
|       강원특별자치도 강릉시 포남동|              대인4|       19.99|  200601|          1000|  5|    1995|
|       강원특별자치도 강릉시 포남동|              대인4|       19.99|  200601|          1850| 10|    1995|
|       강원특별자치도 속초시 조양동|              부영3|     166.535|  200601|         15650| 11|    1996|
|       강원특별자치도 속초시 조양동|              부영3|     133.835|  200601|         12100|  9|    1996|
|강원특별자치도 양양군 양양읍 구교리|    

거래금액(만원) -20500.0 63500.0


number of outliers:  578946
+--------------------------+---------------+------------+--------+--------------+---+--------+
|                    시군구|         단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|
+--------------------------+---------------+------------+--------+--------------+---+--------+
|경기도 고양일산동구 마두동| 강촌마을(영남)|      165.99|  200601|         78000| 18|    1992|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      134.99|  200601|         71300| 10|    1993|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      134.99|  200601|         73000|  5|    1993|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      134.99|  200601|         67000|  7|    1993|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      164.37|  200601|         87500| 17|    1993|
|경기도 고양일산동구 마두동| 강촌마을(우방)|      164.37|  200601|        102750|  5|    1993|
|경기도 고양일산동구 마두동| 강촌마을(한신)|      173.58|  200601|         78500| 17|    1993|
|경기도 고양일산동구 마두동| 강촌마을(한신)|      173.58|  200601|         73500| 12|    1993|
|경기도 고양일산동구 마두동| 강촌마을(한신)|     134.803|  200601|         67500| 10|    1993|
|경기도 고양일산동구 마두동| 백마마

# 레이블 인코딩하기 StringIndexer
* 데이터 프레임의 데이터 타입을 검출한 후 string이면 StringIndexer를 사용해 숫자로 변환

In [51]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="단지명", outputCol="apart_name_Index")
indexed = indexer.fit(data).transform(data)

In [52]:
indexed.select("apart_name_Index").dtypes

[('apart_name_Index', 'double')]

In [53]:
indexed.show()

23/11/30 14:12:49 WARN DAGScheduler: Broadcasting large task binary with size 1456.5 KiB
+---------------------------------+-----------+------------+--------+--------------+---+--------+----------------+
|                           시군구|     단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|apart_name_Index|
+---------------------------------+-----------+------------+--------+--------------+---+--------+----------------+
|강원특별자치도 양구군 양구읍 상리|       경림|        59.4|  200509|          5000| 11|    1998|          2139.0|
|           경기도 의정부시 민락동| 한라비발디|       84.99|  200510|         12750| 19|    2003|            25.0|
|           경기도 의정부시 신곡동|      신일1|      59.878|  200510|          5750|  3|    1997|          6011.0|
|           경기도 의정부시 신곡동|       풍림|       49.83|  200510|          5500|  8|    1998|            36.0|
|           경기도 의정부시 용현동|용현현대1차|      129.73|  200511|         12200| 11|    1992|          2206.0|
|           경기도 의정부시 용현동|용현현대1차|      129.73|  200511|         11500| 17|    1992|      